In [1]:
import pandas as pd

DB_FILE = "sqlite:///sqlite_db_pythonsqlite.db"

Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.

In [2]:
pd.read_sql_query("""
                  SELECT * 
                  FROM Facilities 
                  WHERE membercost <> 0;
                  """, DB_FILE)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,4,Massage Room 1,9.9,80.0,4000,3000
3,5,Massage Room 2,9.9,80.0,4000,3000
4,6,Squash Court,3.5,17.5,5000,80


Q2: How many facilities do not charge a fee to members?

In [3]:
pd.read_sql_query("""
                  SELECT *
                  FROM Facilities
                  WHERE membercost = 0;
                  """, DB_FILE)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,2,Badminton Court,0,15.5,4000,50
1,3,Table Tennis,0,5.0,320,10
2,7,Snooker Table,0,5.0,450,15
3,8,Pool Table,0,5.0,400,15


Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [4]:
pd.read_sql_query("""
                  SELECT facid, name, membercost, monthlymaintenance
                  FROM Facilities
                  WHERE (membercost / monthlymaintenance) < .2;
                  """, DB_FILE)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,2,Badminton Court,0.0,50
3,3,Table Tennis,0.0,10
4,4,Massage Room 1,9.9,3000
5,5,Massage Room 2,9.9,3000
6,6,Squash Court,3.5,80
7,7,Snooker Table,0.0,15
8,8,Pool Table,0.0,15


Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

In [5]:
pd.read_sql_query("""
                  SELECT *
                  FROM Facilities
                  WHERE facid IN (1, 5);
                  """, DB_FILE)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

In [6]:
pd.read_sql_query("""
                  SELECT name, monthlymaintenance,
                    CASE WHEN monthlymaintenance > 100 THEN 'expensive'
                    ELSE 'cheap'
                  END AS is_cheap
                  FROM Facilities;
                  """, DB_FILE)

,name,monthlymaintenance,is_cheap
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

In [7]:
pd.read_sql_query("""
                  SELECT firstname, surname, MAX(joindate) AS joined
                  FROM Members
                  """, DB_FILE)

,firstname,surname,joined
0,Darren,Smith,2012-09-26 18:08:45


Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [31]:
pd.read_sql_query("""
                  SELECT DISTINCT b.bookid, b.facid, f.name, m.firstname || ' ' || m.surname AS member_name
                  FROM Bookings AS b
                  LEFT JOIN Members AS m
                    using(memid)
                  LEFT JOIN Facilities AS f
                    using(facid)
                  WHERE facid <= 1
                  ORDER BY member_name;
                  """, DB_FILE)

,bookid,facid,name,member_name
0,1057,0,Tennis Court 1,Anne Baker
1,1142,1,Tennis Court 2,Anne Baker
2,1240,1,Tennis Court 2,Anne Baker
3,1242,1,Tennis Court 2,Anne Baker
4,1293,1,Tennis Court 2,Anne Baker
...,...,...,...,...
792,1701,0,Tennis Court 1,Tracy Smith
793,2076,0,Tennis Court 1,Tracy Smith
794,2356,0,Tennis Court 1,Tracy Smith
795,3496,0,Tennis Court 1,Tracy Smith


Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [46]:
pd.read_sql_query("""
                  SELECT b.starttime, f.name, m.firstname || ' ' || m.surname AS member_name, f.guestcost
                  FROM Bookings AS b
                  LEFT JOIN Members as m
                    using(memid)
                  LEFT JOIN Facilities as f
                    using(facid)
                  WHERE b.starttime LIKE '2012-09-14 %'
                    AND memid = 0
                    AND f.guestcost > 30
                  ORDER BY f.guestcost DESC
                  """, DB_FILE)

,starttime,name,member_name,guestcost
0,2012-09-14 09:00:00,Massage Room 1,GUEST GUEST,80
1,2012-09-14 13:00:00,Massage Room 1,GUEST GUEST,80
2,2012-09-14 16:00:00,Massage Room 1,GUEST GUEST,80
3,2012-09-14 11:00:00,Massage Room 2,GUEST GUEST,80


Q9: This time, produce the same result as in Q8, but using a subquery.

In [56]:
pd.read_sql_query("""
                  SELECT b.starttime, f.name, member_name, f.guestcost
                  FROM Bookings AS b
                  LEFT JOIN (
                    SELECT memid, firstname || ' ' || surname AS member_name
                    FROM Members
                  )
                  USING (memid)
                  Left JOIN Facilities as f
                    USING (facid)
                  WHERE b.starttime LIKE '2012-09-14 %'
                    AND memid = 0
                    AND f.guestcost > 30
                  ORDER BY f.guestcost DESC
                  """, DB_FILE)

,starttime,name,member_name,guestcost
0,2012-09-14 09:00:00,Massage Room 1,GUEST GUEST,80
1,2012-09-14 13:00:00,Massage Room 1,GUEST GUEST,80
2,2012-09-14 16:00:00,Massage Room 1,GUEST GUEST,80
3,2012-09-14 11:00:00,Massage Room 2,GUEST GUEST,80


Q10: Produce a list of facilities with a total revenue less than 1000.

In [116]:
# go through and calculate the number of times a facility was used by a member
# go through and calculate the number of times a facitlity was used by a guest
# multiply counts by prices and combine
# filter revenue for anything under $1000

pd.read_sql_query("""
                  SELECT facid, member_usage, guest_usage, (member_usage * membercost) + (guest_usage * guestcost) AS revenue
                  FROM Facilities
                  LEFT OUTER JOIN
                  (
                    SELECT facid, COUNT(*) AS member_usage
                    FROM Bookings
                    WHERE memid <> 0
                    GROUP BY facid
                  ) USING(facid)
                  LEFT OUTER JOIN
                  (
                    SELECT facid, COUNT(*) AS guest_usage
                    FROM Bookings
                    WHERE memid = 0
                    GROUP BY facid
                  ) USING (facid)
                  WHERE revenue < 1000
                  """, DB_FILE)

,facid,member_usage,guest_usage,revenue
0,2,344,39,604.5
1,3,385,18,90.0
2,7,421,23,115.0
3,8,783,53,265.0


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [133]:
pd.read_sql_query("""
                  SELECT surname || ', ' || firstname AS member, name AS recommended_by
                  FROM Members AS m1
                  LEFT JOIN (
                    SELECT memid, firstname || ' ' || surname AS name
                    FROM Members
                  ) AS m2 ON m1.recommendedby = m2.memid
                  WHERE recommendedby IS NOT ""
                  ORDER BY surname, firstname
                  """, DB_FILE)

,member,recommended_by
0,"Bader, Florence",Ponder Stibbons
1,"Baker, Anne",Ponder Stibbons
2,"Baker, Timothy",Jemima Farrell
3,"Boothe, Tim",Tim Rownam
4,"Butters, Gerald",Darren Smith
5,"Coplin, Joan",Timothy Baker
6,"Crumpet, Erica",Tracy Smith
7,"Dare, Nancy",Janice Joplette
8,"Genting, Matthew",Gerald Butters
9,"Hunt, John",Millicent Purview


Q12: Find the facilities with their usage by member, but not guests 

In [139]:
pd.read_sql_query("""
                  SELECT name, member_usage
                  FROM Facilities
                  LEFT OUTER JOIN
                  (
                    SELECT facid, COUNT(*) AS member_usage
                    FROM Bookings
                    WHERE memid <> 0
                    GROUP BY facid
                  ) USING(facid)
                  GROUP BY facid
                  ORDER BY member_usage DESC
                  """, DB_FILE)

,name,member_usage
0,Pool Table,783
1,Massage Room 1,421
2,Snooker Table,421
3,Table Tennis,385
4,Badminton Court,344
5,Tennis Court 1,308
6,Tennis Court 2,276
7,Squash Court,195
8,Massage Room 2,27


Q13: Find the facilities usage by month, but not guests

In [162]:
pd.read_sql_query("""
                  SELECT name, SUBSTRING(starttime, 1, LENGTH(starttime) - 12) AS date, COUNT(slots)
                  FROM Bookings
                  LEFT JOIN (
                    SELECT facid, name
                    FROM Facilities
                  ) USING (facid)
                  WHERE memid <> 0
                  GROUP BY facid, date
                  """, DB_FILE)

,name,date,COUNT(slots)
0,Tennis Court 1,2012-07,65
1,Tennis Court 1,2012-08,111
2,Tennis Court 1,2012-09,132
3,Tennis Court 2,2012-07,41
4,Tennis Court 2,2012-08,109
5,Tennis Court 2,2012-09,126
6,Badminton Court,2012-07,51
7,Badminton Court,2012-08,132
8,Badminton Court,2012-09,161
9,Table Tennis,2012-07,48
